In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class KITTIDataset(Dataset):
    def __init__(self, images_dir, masks_dir=None, transform=None, mask_transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.mask_transform = mask_transform
        self.image_names = []
        self.mask_names = []
        
        if masks_dir:
            
            for img_name in os.listdir(images_dir):
                base_name1, base_name2 = img_name.split("_")
                
                mask_name = f"{base_name1}_lane_{base_name2}"
                
                if os.path.exists(os.path.join(masks_dir, mask_name)):
                    self.image_names.append(img_name)
                    self.mask_names.append(mask_name)
        else:
            
            self.image_names = os.listdir(images_dir)
    
        
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        if self.masks_dir:
            mask_name = self.mask_names[idx]
            mask_path = os.path.join(self.masks_dir, mask_name)
            mask = Image.open(mask_path).convert("L")  
            if self.mask_transform:
                mask = self.mask_transform(mask)
            return image, mask
        else:
            return image, img_name  


image_transforms = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  
                         std=[0.229, 0.224, 0.225])
])

mask_transforms = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor()
])


train_dataset = KITTIDataset(
    images_dir="dataset/data_road/training/image_2",
    masks_dir="dataset/data_road/training/gt_image_2",
    transform=image_transforms,
    mask_transform=mask_transforms
)

test_dataset = KITTIDataset(
    images_dir="dataset/data_road/testing/image_2",
    masks_dir=None,
    transform=image_transforms
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)


In [ ]:
class UNet(nn.Module):
    def __init__(self, n_classes=1):
        super(UNet, self).__init__()
        
        def CBR(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        self.enc1 = CBR(3, 64)
        self.enc2 = CBR(64, 128)
        self.enc3 = CBR(128, 256)
        self.enc4 = CBR(256, 512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.center = CBR(512, 1024)
        
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = CBR(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = CBR(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = CBR(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = CBR(128, 64)
        
        self.final = nn.Conv2d(64, n_classes, 1)
        
    def forward(self, x):
        enc1 = self.enc1(x)      # [B, 64, H, W]
        enc2 = self.enc2(self.pool(enc1))  # [B, 128, H/2, W/2]
        enc3 = self.enc3(self.pool(enc2))  # [B, 256, H/4, W/4]
        enc4 = self.enc4(self.pool(enc3))  # [B, 512, H/8, W/8]
        
        center = self.center(self.pool(enc4))  # [B, 1024, H/16, W/16]
        
        dec4 = self.up4(center)  # [B, 512, H/8, W/8]
        dec4 = torch.cat([dec4, enc4], dim=1)  # [B, 1024, H/8, W/8]
        dec4 = self.dec4(dec4)  # [B, 512, H/8, W/8]
        
        dec3 = self.up3(dec4)  # [B, 256, H/4, W/4]
        dec3 = torch.cat([dec3, enc3], dim=1)  # [B, 512, H/4, W/4]
        dec3 = self.dec3(dec3)  # [B, 256, H/4, W/4]
        
        dec2 = self.up2(dec3)  # [B, 128, H/2, W/2]
        dec2 = torch.cat([dec2, enc2], dim=1)  # [B, 256, H/2, W/2]
        dec2 = self.dec2(dec2)  # [B, 128, H/2, W/2]
        
        dec1 = self.up1(dec2)  # [B, 64, H, W]
        dec1 = torch.cat([dec1, enc1], dim=1)  # [B, 128, H, W]
        dec1 = self.dec1(dec1)  # [B, 64, H, W]
        
        final = self.final(dec1)  # [B, n_classes, H, W]
        return final
model = UNet(n_classes=1)  


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 200

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, preds, targets):
        preds = preds.view(-1)
        targets = targets.view(-1)
        intersection = (preds * targets).sum()
        dice = (2.*intersection + self.smooth) / (preds.sum() + targets.sum() + self.smooth)
        return 1 - dice

criterion = nn.BCEWithLogitsLoss()



In [ ]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}") as pbar:
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device)
            masks = masks.float()  
            masks = (masks > 100/255).float()  
            
            optimizer.zero_grad()
            outputs = model(images)
            outputs = outputs.squeeze(1)  
            loss = criterion(outputs, masks.squeeze(1))
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            pbar.set_postfix({'loss': epoch_loss / (pbar.n + 1)})
            pbar.update(1)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")
    
    
    if epoch == num_epochs - 1 or (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f"unet_epoch_{epoch+1}.pth")


Epoch 1/200: 100%|██████████| 6/6 [00:21<00:00,  3.50s/it, loss=0.656]


Epoch 1/200, Loss: 0.6565


Epoch 2/200: 100%|██████████| 6/6 [00:17<00:00,  2.92s/it, loss=0.591]


Epoch 2/200, Loss: 0.5912


Epoch 3/200: 100%|██████████| 6/6 [00:17<00:00,  2.92s/it, loss=0.521]


...

Epoch 198/200, Loss: 0.0429


Epoch 199/200: 100%|██████████| 6/6 [00:29<00:00,  4.88s/it, loss=0.0424]


Epoch 199/200, Loss: 0.0424


Epoch 200/200: 100%|██████████| 6/6 [00:28<00:00,  4.76s/it, loss=0.042] 

Epoch 200/200, Loss: 0.0420


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    original_size = image.size  
    image = transform(image).unsqueeze(0).to(device)  
    return image, original_size

def postprocess_output(output, original_size, threshold=0.5):
    output = output.squeeze().cpu().detach().numpy()
    output = sigmoid(output)
    output = (output > threshold).astype(np.uint8) * 255
    output = Image.fromarray(output).resize(original_size, Image.NEAREST)
    return np.array(output)

def hough_transform(image, threshold=50, min_line_length=50, max_line_gap=150):
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold, 
                            minLineLength=min_line_length, 
                            maxLineGap=max_line_gap)
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(line_image, (x1,y1), (x2,y2), 255, 2)
    return line_image

def combine_segmentation_hough(segmentation, hough):
    combined = cv2.bitwise_and(segmentation, hough)
    return combined

def hough_transform_area(image, threshold=50, min_line_length=50, max_line_gap=150):
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold, 
                            minLineLength=min_line_length, 
                            maxLineGap=max_line_gap)
    line_image = np.zeros_like(image)
    points = []
    
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                
                points.append([x1, y1])
                points.append([x2, y2])
                
                cv2.line(line_image, (x1, y1), (x2, y2), 255, 2)
    
    if points:
        
        points = np.array(points)
        hull = cv2.convexHull(points)
        area_mask = np.zeros_like(image)
        cv2.fillConvexPoly(area_mask, hull, 255)
        return area_mask
    else:
        
        return np.zeros_like(image)

def visualize_result(original_image_path, segmentation, combined, save_path=None):
    original = cv2.imread(original_image_path)
    original = cv2.resize(original, (segmentation.shape[1], segmentation.shape[0]))
    
    overlay = original.copy()
    overlay[segmentation > 0] = [0, 255, 0]

    
    cv2.addWeighted(original, 0.7, overlay, 0.3, 0, overlay)
    
    if save_path:
        cv2.imwrite(save_path, overlay)
    else:
        cv2.imshow("Lane Detection", overlay)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
def keep_largest_connected_area(mask):
    if mask.dtype != np.uint8:
        mask = mask.astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return np.zeros_like(mask)

    
    largest_contour = max(contours, key=cv2.contourArea)

    
    largest_area_mask = np.zeros_like(mask)
    cv2.drawContours(largest_area_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    return largest_area_mask


In [ ]:
def process_video(input_video_path, output_video_path, transform, model, device):
    cap = cv2.VideoCapture(input_video_path)
    
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        
        
        temp_image_path = "temp_frame.png"
        cv2.imwrite(temp_image_path, frame)
        
        
        image, original_size = preprocess_image(temp_image_path, transform)
        
        
        model.eval()
        with torch.no_grad():
            output = model(image.to(device))
        segmentation = postprocess_output(output, original_size)

        original_image = cv2.resize(frame, (segmentation.shape[1], segmentation.shape[0]))
        overlay = original_image.copy()
        overlay[segmentation > 0] = [0, 255, 0]  
        
        
        cv2.addWeighted(original_image, 0.7, overlay, 0.3, 0, overlay)
        
        
        final_frame = cv2.resize(overlay, (frame_width, frame_height))
        
        
        out.write(final_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    if os.path.exists("temp_frame.png"):
        os.remove("temp_frame.png")



In [ ]:
# model.load_state_dict(torch.load(f"unet_epoch_{200}.pth"))
model.eval()



C:\Users\11439\AppData\Local\Temp\ipykernel_108596\1282795143.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"unet_epoch_{200}.pth"))


UNet(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (center): Sequential(
    (0

In [ ]:

for test_img in os.listdir("dataset/data_road/testing/image_2"):
    test_image_path = "dataset/data_road/testing/image_2/" + test_img
    
    image, original_size = preprocess_image(test_image_path, image_transforms)
    with torch.no_grad():
        output = model(image.to(device))
    segmentation = postprocess_output(output, original_size)
    segmentation = keep_largest_connected_area(segmentation)
    hough = hough_transform_area(segmentation)
    combined = combine_segmentation_hough(segmentation, hough)
    if not os.path.exists("results"):
        os.mkdir("results")
    visualize_result(test_image_path, segmentation, combined, save_path=f"results/{test_img}")


In [ ]:
input_video_path = "project_video.mp4"  
output_video_path = "project_video_detected.avi"
process_video(input_video_path, output_video_path, image_transforms, model, device)

In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    dice_scores = []
    iou_scores = []
    pixel_accuracies = []
    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = (masks>100/255).to(device).float()
            outputs = model(images)
            outputs = torch.sigmoid(outputs)
            outputs = (outputs > 0.5).float()

            
            outputs_flat = outputs.view(-1)
            masks_flat = masks.view(-1)

            intersection = (outputs_flat * masks_flat).sum()
            dice_score = (2. * intersection + 1e-6) / (outputs_flat.sum() + masks_flat.sum() + 1e-6)
            dice_scores.append(dice_score.item())
      
            union = outputs_flat.sum() + masks_flat.sum() - intersection
            iou_score = (intersection + 1e-6) / (union + 1e-6)
            iou_scores.append(iou_score.item())

            
            correct = (outputs_flat == masks_flat).sum()
            total = outputs_flat.numel()
            pixel_accuracy = correct.item() / total
            pixel_accuracies.append(pixel_accuracy)

    avg_dice = np.mean(dice_scores)
    avg_iou = np.mean(iou_scores)
    avg_accuracy = np.mean(pixel_accuracies)

    print(f"Average Dice Coefficient: {avg_dice:.4f}")
    print(f"Average IoU: {avg_iou:.4f}")
    print(f"Average Pixel Accuracy: {avg_accuracy:.4f}")

    return avg_dice, avg_iou, avg_accuracy

evaluate(model, train_loader, device)

Average Dice Coefficient: 0.9981
Average IoU: 0.9962
Average Pixel Accuracy: 0.9997


(np.float64(0.9980734884738922),
 np.float64(0.9961545169353485),
 np.float64(0.9996948877970379))